# Import et chargement des données

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload  

import kagfonct as kf


In [2]:
# NB : utile pour relaoder le module kf si je fais des modifications dedans
# reload(kf)

In [3]:
PATH = "./data/"
for fic in os.listdir(PATH):
    print(fic)
# print(os.listdir(path))   

application_test.csv
application_train.csv
bureau.csv
bureau_balance.csv
credit_card_balance.csv
HomeCredit_columns_description - Copy.csv
HomeCredit_columns_description - test.csv
HomeCredit_columns_description - utf8.csv
HomeCredit_columns_description.csv
HomeCredit_columns_description.ods
installments_payments.csv
POS_CASH_balance.csv
previous_application.csv
Projet+Mise+en+prod+-+home-credit-default-risk.zip
sample_submission.csv


In [4]:
df_application_test = pd.read_csv(PATH+"application_test.csv")
df_application_train = pd.read_csv(PATH+"application_train.csv")
df_bureau = pd.read_csv(PATH+"bureau.csv")
df_bureau_balance = pd.read_csv(PATH+"bureau_balance.csv")
df_credit_card_balance = pd.read_csv(PATH+"credit_card_balance.csv")
df_payments = pd.read_csv(PATH+"installments_payments.csv")
df_pos_cash_balance = pd.read_csv(PATH+"POS_CASH_balance.csv")
df_prev_app = pd.read_csv(PATH+"previous_application.csv")
# df_homecredit = pd.read_csv(PATH+"HomeCredit_columns_description - utf8.csv")
df_homecredit = pd.read_csv(PATH+"HomeCredit_columns_description.csv", encoding = "cp1252")

NB : le fichier "HomeCredit_columns_description.csv" comporte à plusieurs reprises, un caractère qui n'est pas dans la norme utf-8 (…)

In [5]:
df_homecredit.loc[12,"Description"]

'Clients income type (businessman, working, maternity leave,…)'

# Fonctions

# Exploration

In [6]:
df_application_train.describe()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,307511.000000,307511.000000,307511.000000,3.075110e+05,3.075110e+05,307499.000000,3.072330e+05,307511.000000,307511.000000,307511.000000,...,307511.000000,307511.000000,307511.000000,307511.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000
mean,278180.518577,0.080729,0.417052,1.687979e+05,5.990260e+05,27108.573909,5.383962e+05,0.020868,-16036.995067,63815.045904,...,0.008130,0.000595,0.000507,0.000335,0.006402,0.007000,0.034362,0.267395,0.265474,1.899974
std,102790.175348,0.272419,0.722121,2.371231e+05,4.024908e+05,14493.737315,3.694465e+05,0.013831,4363.988632,141275.766519,...,0.089798,0.024387,0.022518,0.018299,0.083849,0.110757,0.204685,0.916002,0.794056,1.869295
min,100002.000000,0.000000,0.000000,2.565000e+04,4.500000e+04,1615.500000,4.050000e+04,0.000290,-25229.000000,-17912.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,189145.500000,0.000000,0.000000,1.125000e+05,2.700000e+05,16524.000000,2.385000e+05,0.010006,-19682.000000,-2760.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,278202.000000,0.000000,0.000000,1.471500e+05,5.135310e+05,24903.000000,4.500000e+05,0.018850,-15750.000000,-1213.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,367142.500000,0.000000,1.000000,2.025000e+05,8.086500e+05,34596.000000,6.795000e+05,0.028663,-12413.000000,-289.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,456255.000000,1.000000,19.000000,1.170000e+08,4.050000e+06,258025.500000,4.050000e+06,0.072508,-7489.000000,365243.000000,...,1.000000,1.000000,1.000000,1.000000,4.000000,9.000000,8.000000,27.000000,261.000000,25.000000


In [7]:
df_application_train.sample(10)

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
158676,283955,0,Cash loans,F,N,N,1,112500.0,495000.0,26982.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
117053,235732,1,Cash loans,M,N,N,1,135000.0,450000.0,21888.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
77561,189928,0,Revolving loans,F,Y,Y,0,112500.0,270000.0,13500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
186959,316734,0,Cash loans,M,Y,Y,1,202500.0,450000.0,36211.5,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,1.0
58456,167761,0,Cash loans,F,N,Y,0,135000.0,675000.0,24043.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
175723,303627,0,Cash loans,F,Y,Y,0,108000.0,153576.0,9945.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0
145601,268826,0,Cash loans,M,N,Y,0,76500.0,675000.0,26770.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
96368,211878,0,Cash loans,F,N,Y,0,157500.0,297130.5,13216.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
156967,281933,0,Revolving loans,M,Y,Y,0,126000.0,382500.0,19125.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
113395,231508,0,Cash loans,M,Y,Y,0,76500.0,745429.5,24763.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df_homecredit[df_homecredit["Table"]=="application_{train|test}.csv"]

,Unnamed: 0,Table,Row,Description,Special
0,1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
1,2,application_{train|test}.csv,TARGET,Target variable (1 - client with payment diffi...,NaN
2,5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
4,7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN
...,...,...,...,...,...
117,120,application_{train|test}.csv,AMT_REQ_CREDIT_BUREAU_DAY,Number of enquiries to Credit Bureau about the...,NaN
118,121,application_{train|test}.csv,AMT_REQ_CREDIT_BUREAU_WEEK,Number of enquiries to Credit Bureau about the...,NaN
119,122,application_{train|test}.csv,AMT_REQ_CREDIT_BUREAU_MON,Number of enquiries to Credit Bureau about the...,NaN
120,123,application_{train|test}.csv,AMT_REQ_CREDIT_BUREAU_QRT,Number of enquiries to Credit Bureau about the...,NaN


In [9]:
list(df_homecredit.loc[(df_homecredit["Table"]=="application_{train|test}.csv") & (df_homecredit["Row"]=="TARGET"), 
                       "Description"])

['Target variable (1 - client with payment difficulties: he/she had late payment more than X days on at least one of the first Y installments of the loan in our sample, 0 - all other cases)']

In [10]:
list(df_application_train.columns)

['SK_ID_CURR',
 'TARGET',
 'NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'OCCUPATION_TYPE',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'WEEKDAY_APPR_PROCESS_START',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'ORGANIZATION_TYPE',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_A

In [11]:
df_application_test.sample(10)

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
12689,192423,Cash loans,F,N,Y,0,157500.0,1579500.0,55030.5,1579500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
27901,303607,Cash loans,M,N,Y,0,270000.0,848745.0,43465.5,675000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,7.0
20861,251336,Cash loans,F,N,N,1,180000.0,675000.0,53460.0,675000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
13101,195158,Cash loans,M,Y,Y,0,225000.0,599778.0,30753.0,477000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,7.0
24874,281137,Cash loans,F,N,Y,0,135000.0,621900.0,37161.0,562500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,7.0
48310,452974,Cash loans,F,N,N,1,112500.0,900000.0,38133.0,900000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,2.0
7662,156140,Cash loans,M,N,N,0,135000.0,440784.0,34956.0,360000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0
47689,448526,Cash loans,F,Y,Y,0,271651.5,135000.0,13216.5,135000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
21582,256663,Cash loans,F,Y,N,0,360000.0,383760.0,38088.0,360000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
9901,171891,Cash loans,F,N,Y,0,135000.0,481176.0,24696.0,360000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0


In [12]:
df_bureau.shape

(1716428, 17)

In [13]:
df_bureau.sample(10)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
143527,303401,6561243,Closed,currency 1,-2881,0,-2485.0,-2435.0,16255.215,0,51135.30,0.0,0.0,0.0,Consumer credit,-2423,NaN
1160665,222928,5907057,Active,currency 1,-79,0,1017.0,NaN,NaN,0,472986.00,464746.5,0.0,0.0,Consumer credit,-46,20052.0
509529,294479,5137086,Active,currency 1,-822,0,31059.0,NaN,NaN,0,81000.00,76752.0,NaN,0.0,Credit card,-18,0.0
1480468,190369,5353888,Closed,currency 1,-1164,0,-798.0,-804.0,NaN,0,85455.00,0.0,0.0,0.0,Consumer credit,-792,0.0
851994,144175,6055387,Closed,currency 1,-1555,0,271.0,-352.0,NaN,0,391500.00,0.0,0.0,0.0,Consumer credit,-136,NaN
1284576,351615,5657117,Closed,currency 1,-1392,0,-1027.0,-1027.0,NaN,0,58500.00,0.0,0.0,0.0,Consumer credit,-726,NaN
1310681,208667,5433292,Closed,currency 1,-2496,0,-2313.0,-2310.0,0.000,0,62154.00,NaN,NaN,0.0,Consumer credit,-2305,NaN
1034286,437890,6770431,Closed,currency 1,-1782,0,-1570.0,-1598.0,0.000,0,33106.32,0.0,0.0,0.0,Consumer credit,-1598,NaN
862823,213933,6045027,Active,currency 1,-2146,0,1507.0,NaN,NaN,0,243000.00,25218.0,0.0,0.0,Credit card,-18,NaN
1176601,348119,5953250,Closed,currency 1,-923,0,781.0,-527.0,NaN,0,3518550.00,0.0,0.0,0.0,Car loan,-510,1857595.5


In [14]:
df_bureau[df_bureau["SK_ID_CURR"]==368092]

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
514934,368092,6457783,Closed,currency 1,-487,0,-122.0,-121.0,NaN,0,114750.0,0.0,0.0,0.0,Consumer credit,-116,0.0
514935,368092,6457784,Closed,currency 1,-1676,0,-580.0,-1505.0,NaN,0,270000.0,0.0,0.0,0.0,Consumer credit,-1423,5044.5
514936,368092,6457785,Closed,currency 1,-1676,0,-580.0,-610.0,NaN,0,270000.0,0.0,0.0,0.0,Consumer credit,-339,0.0
514937,368092,6457786,Closed,currency 1,-1819,0,-1547.0,-1197.0,NaN,0,83385.0,NaN,NaN,0.0,Consumer credit,-1194,0.0


In [15]:
df_bureau_balance[df_bureau_balance["SK_ID_BUREAU"]== 6457783]

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
10396161,6457783,0,C
10396162,6457783,-1,C
10396163,6457783,-2,C
10396164,6457783,-3,C
10396165,6457783,-4,C
10396166,6457783,-5,0
10396167,6457783,-6,0
10396168,6457783,-7,0
10396169,6457783,-8,X
10396170,6457783,-9,0


In [16]:
df_bureau_balance["STATUS"].unique()

array(['C', '0', 'X', '1', '2', '3', '5', '4'], dtype=object)

In [17]:
df_homecredit[df_homecredit["Table"]=="bureau_balance.csv"]

,Unnamed: 0,Table,Row,Description,Special
139,142,bureau_balance.csv,SK_BUREAU_ID,Recoded ID of Credit Bureau credit (unique cod...,hashed
140,143,bureau_balance.csv,MONTHS_BALANCE,Month of balance relative to application date ...,time only relative to the application
141,144,bureau_balance.csv,STATUS,Status of Credit Bureau loan during the month ...,NaN


In [18]:
df_pos_cash_balance.shape

(10001358, 8)

In [19]:
df_pos_cash_balance[df_pos_cash_balance["SK_ID_CURR"]==368092].sort_values(by="MONTHS_BALANCE")

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
9535021,2198792,368092,-82,12.0,12.0,Active,0,0
948279,2198792,368092,-81,12.0,11.0,Active,0,0
9001077,2198792,368092,-80,12.0,10.0,Active,0,0
6410934,2198792,368092,-79,12.0,9.0,Active,0,0
4157178,2198792,368092,-78,12.0,8.0,Active,0,0
4097603,2198792,368092,-77,12.0,7.0,Active,0,0
1377294,2198792,368092,-76,12.0,6.0,Active,0,0
2462432,2198792,368092,-75,12.0,5.0,Active,0,0
3171983,2198792,368092,-74,12.0,4.0,Active,0,0
3216998,2198792,368092,-73,12.0,3.0,Active,0,0


In [20]:
df_homecredit[df_homecredit["Table"]=="POS_CASH_balance.csv"]

,Unnamed: 0,Table,Row,Description,Special
142,145,POS_CASH_balance.csv,SK_ID_PREV,ID of previous credit in Home Credit related t...,NaN
143,146,POS_CASH_balance.csv,SK_ID_CURR,ID of loan in our sample,NaN
144,147,POS_CASH_balance.csv,MONTHS_BALANCE,Month of balance relative to application date ...,time only relative to the application
145,148,POS_CASH_balance.csv,CNT_INSTALMENT,Term of previous credit (can change over time),NaN
146,149,POS_CASH_balance.csv,CNT_INSTALMENT_FUTURE,Installments left to pay on the previous credit,NaN
147,150,POS_CASH_balance.csv,NAME_CONTRACT_STATUS,Contract status during the month,NaN
148,151,POS_CASH_balance.csv,SK_DPD,DPD (days past due) during the month of previo...,NaN
149,152,POS_CASH_balance.csv,SK_DPD_DEF,DPD during the month with tolerance (debts wit...,NaN


In [21]:
df_credit_card_balance.sample(10)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
604614,2208094,441760,-22,0.000,135000,0.0,0.0,0.0,0.0,0.000,...,0.000,0.000,0.0,0,0.0,0.0,34.0,Completed,0,0
3161375,2190419,363949,-58,137981.655,135000,22500.0,22500.0,0.0,0.0,6750.000,...,137981.655,137981.655,1.0,1,0.0,0.0,13.0,Active,0,0
3549757,1290148,430314,-22,0.000,90000,0.0,0.0,0.0,0.0,0.000,...,0.000,0.000,0.0,0,0.0,0.0,49.0,Active,0,0
614544,2242301,413886,-6,0.000,180000,0.0,0.0,0.0,0.0,0.000,...,0.000,0.000,0.0,0,0.0,0.0,36.0,Completed,0,0
2710479,1405398,361604,-13,61286.490,67500,0.0,0.0,0.0,0.0,3375.000,...,61286.490,61286.490,0.0,0,0.0,0.0,30.0,Active,0,0
3219284,2220401,204740,-58,0.000,0,NaN,0.0,NaN,NaN,NaN,...,0.000,0.000,NaN,0,NaN,NaN,NaN,Active,0,0
2105810,2552982,118154,-12,0.000,0,NaN,0.0,NaN,NaN,0.000,...,0.000,0.000,NaN,0,NaN,NaN,0.0,Active,0,0
306032,2701291,399433,-85,0.000,135000,0.0,0.0,0.0,0.0,0.000,...,0.000,0.000,0.0,0,0.0,0.0,22.0,Active,0,0
727047,2703054,396051,-31,127908.090,135000,13500.0,13500.0,0.0,0.0,6750.000,...,127908.090,127908.090,1.0,1,0.0,0.0,47.0,Active,0,0
562773,1380682,369512,-15,155958.750,450000,31500.0,31500.0,0.0,0.0,6912.315,...,152133.930,152133.930,2.0,2,0.0,0.0,14.0,Active,0,0


In [22]:
df_prev_app.sample(10)

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
43787,1964263,283842,Cash loans,29576.745,585000.0,654498.0,NaN,585000.0,THURSDAY,16,...,XNA,36.0,middle,Cash X-Sell: middle,365243.0,-1019.0,31.0,-959.0,-955.0,1.0
1172025,1143656,211040,Consumer loans,14395.815,145656.0,157194.0,0.0,145656.0,SATURDAY,14,...,Jewelry,12.0,low_action,POS others without interest,365243.0,-3.0,327.0,365243.0,365243.0,0.0
1665872,2763462,113526,Consumer loans,6517.845,58495.5,64674.0,0.0,58495.5,SUNDAY,16,...,Consumer electronics,12.0,middle,POS household with interest,365243.0,-281.0,49.0,365243.0,365243.0,0.0
95813,2597877,211776,Consumer loans,7124.040,66555.0,66226.5,6655.5,66555.0,SUNDAY,17,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-932.0,-602.0,-602.0,-599.0,0.0
1282310,1972413,363837,Revolving loans,NaN,0.0,0.0,NaN,NaN,TUESDAY,11,...,XNA,NaN,XNA,Card Street,NaN,NaN,NaN,NaN,NaN,NaN
1475638,1824023,431594,Revolving loans,4500.000,0.0,90000.0,NaN,NaN,FRIDAY,11,...,Connectivity,0.0,XNA,Card Street,-2774.0,-2730.0,365243.0,-2242.0,365243.0,0.0
199077,1387455,187293,Cash loans,46296.630,405000.0,475227.0,NaN,405000.0,WEDNESDAY,18,...,XNA,12.0,low_normal,Cash X-Sell: low,365243.0,-556.0,-226.0,-526.0,-519.0,1.0
910370,2740618,289881,Cash loans,NaN,0.0,0.0,NaN,NaN,FRIDAY,7,...,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
1235812,1173323,171090,Cash loans,NaN,0.0,0.0,NaN,NaN,SUNDAY,6,...,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
1478316,2704726,243034,Consumer loans,10735.920,171436.5,180769.5,17145.0,171436.5,TUESDAY,7,...,Consumer electronics,24.0,middle,POS household with interest,365243.0,-192.0,498.0,365243.0,365243.0,1.0


In [23]:
df_prev_app[df_prev_app["SK_ID_CURR"]==368092]

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
132593,2198792,368092,Consumer loans,5921.37,122764.14,60732.0,67504.14,122764.14,MONDAY,13,...,Consumer electronics,12.0,middle,POS household with interest,365243.0,-2475.0,-2145.0,-2145.0,-2136.0,1.0


In [24]:
df_payments.shape

(13605401, 8)

In [25]:
df_payments.sample(10)

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
7604486,2077999,260244,1.0,2,-2634.0,-2634.0,5645.205,5645.205
7110546,1541130,258054,1.0,3,-823.0,-837.0,11251.845,11251.845
184327,1687733,103073,0.0,65,-1357.0,-1357.0,113.130,113.130
10429836,1024866,355540,1.0,9,-561.0,-576.0,10125.810,10125.810
7497050,1153914,215234,1.0,6,-1900.0,-1933.0,4346.370,4346.370
4252015,1782456,246946,1.0,12,-615.0,-608.0,41360.895,39111.345
7531891,1716957,275086,0.0,4,-170.0,-170.0,2250.000,2250.000
8861185,2137843,322515,1.0,5,-2676.0,-2676.0,2092.275,2092.275
9018564,1082517,364731,1.0,2,-596.0,-603.0,7499.250,7499.250
12281890,1895178,402417,1.0,11,-1280.0,-1285.0,14240.520,14240.520


In [26]:
df_payments[df_payments["SK_ID_CURR"]==368092].sort_values(by="NUM_INSTALMENT_NUMBER")

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
8624343,2198792,368092,1.0,1,-2475.0,-2485.0,5921.37,5921.37
8085854,2198792,368092,1.0,2,-2445.0,-2447.0,5921.37,5921.37
9254042,2198792,368092,1.0,3,-2415.0,-2419.0,5921.37,5921.37
7829329,2198792,368092,1.0,4,-2385.0,-2387.0,5921.37,5921.37
9254285,2198792,368092,1.0,5,-2355.0,-2358.0,5921.37,5921.37
10778313,2198792,368092,1.0,6,-2325.0,-2344.0,5921.37,5921.37
9235293,2198792,368092,1.0,7,-2295.0,-2296.0,5921.37,5921.37
8320158,2198792,368092,1.0,8,-2265.0,-2274.0,5921.37,5921.37
11155326,2198792,368092,1.0,9,-2235.0,-2246.0,5921.37,5921.37
11034595,2198792,368092,1.0,10,-2205.0,-2204.0,5921.37,5921.37


# Feature ingineering

Ici nous allons utiliser les fonctions récupérées sur Kaggle pour faire des aggrégations et pour optimiser l'usage de la mémoire.

In [27]:
df_application_test = kf.convert_types(df_application_test, print_info=True)
df_application_train = kf.convert_types(df_application_train, print_info=True)
df_bureau = kf.convert_types(df_bureau, print_info=True)
df_bureau_balance = kf.convert_types(df_bureau_balance, print_info=True)
df_credit_card_balance = kf.convert_types(df_credit_card_balance, print_info=True)
df_payments = kf.convert_types(df_payments, print_info=True)
df_pos_cash_balance = kf.convert_types(df_pos_cash_balance, print_info=True)
df_prev_app = kf.convert_types(df_prev_app, print_info=True)


Original Memory Usage: 0.05 gb.
New Memory Usage: 0.03 gb.
Original Memory Usage: 0.3 gb.
New Memory Usage: 0.17 gb.
Original Memory Usage: 0.23 gb.
New Memory Usage: 0.13 gb.
Original Memory Usage: 0.66 gb.
New Memory Usage: 0.35 gb.
Original Memory Usage: 0.71 gb.
New Memory Usage: 0.42 gb.
Original Memory Usage: 0.87 gb.
New Memory Usage: 0.49 gb.
Original Memory Usage: 0.64 gb.
New Memory Usage: 0.41 gb.
Original Memory Usage: 0.49 gb.
New Memory Usage: 0.18 gb.


## tables bureau et bureau_balance

In [28]:
df_bureau_agg = kf.agg_numeric(df_bureau.drop(columns = ['SK_ID_BUREAU']), parent_var = 'SK_ID_CURR', df_name = 'bureau')
df_bureau_agg.head()

,bureau_DAYS_CREDIT_sum,bureau_DAYS_ENDDATE_FACT_sum,bureau_DAYS_CREDIT_min,bureau_DAYS_CREDIT_ENDDATE_min,bureau_DAYS_ENDDATE_FACT_min,bureau_DAYS_ENDDATE_FACT_mean,bureau_DAYS_CREDIT_mean,bureau_DAYS_CREDIT_UPDATE_sum,bureau_DAYS_ENDDATE_FACT_max,bureau_DAYS_CREDIT_UPDATE_min,...,bureau_AMT_CREDIT_SUM_DEBT_mean,bureau_AMT_CREDIT_SUM_min,bureau_AMT_CREDIT_SUM_mean,bureau_AMT_CREDIT_SUM_DEBT_max,bureau_AMT_CREDIT_SUM_max,bureau_AMT_CREDIT_SUM_DEBT_sum,bureau_AMT_CREDIT_SUM_sum,bureau_AMT_CREDIT_MAX_OVERDUE_min,bureau_AMT_CREDIT_MAX_OVERDUE_mean,bureau_AMT_CREDIT_MAX_OVERDUE_max
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-5145,-3302.0,-1572,-1329.0,-1328.0,-825.500000,-735.000000,-652,-544.0,-155,...,85240.929688,85500.0,207623.578125,373239.0,378000.000000,596686.5,1.453365e+06,NaN,NaN,NaN
100002,-6992,-4185.0,-1437,-1072.0,-1185.0,-697.500000,-874.000000,-3999,-36.0,-1185,...,49156.199219,0.0,108131.945312,245781.0,450000.000000,245781.0,8.650556e+05,0.0,1681.029053,5043.64502
100003,-5603,-3292.0,-2586,-2434.0,-2131.0,-1097.333374,-1400.750000,-3264,-540.0,-2131,...,0.000000,22248.0,254350.125000,0.0,810000.000000,0.0,1.017400e+06,0.0,0.000000,0.00000
100004,-1734,-1065.0,-1326,-595.0,-683.0,-532.500000,-867.000000,-1064,-382.0,-682,...,0.000000,94500.0,94518.898438,0.0,94537.796875,0.0,1.890378e+05,0.0,0.000000,0.00000
100005,-572,-123.0,-373,-128.0,-123.0,-123.000000,-190.666667,-163,-123.0,-121,...,189469.500000,29826.0,219042.000000,543087.0,568800.000000,568408.5,6.571260e+05,0.0,0.000000,0.00000


In [29]:
df_bureau_counts = kf.agg_categorical(df_bureau, parent_var = 'SK_ID_CURR', df_name = 'bureau')
df_bureau_counts.head()

,bureau_CREDIT_TYPE_Mobile operator loan_mean,bureau_CREDIT_TYPE_Mobile operator loan_sum,bureau_CREDIT_TYPE_Loan for purchase of shares (margin lending)_mean,bureau_CREDIT_TYPE_Loan for purchase of shares (margin lending)_sum,bureau_CREDIT_ACTIVE_Bad debt_mean,bureau_CREDIT_ACTIVE_Bad debt_sum,bureau_CREDIT_TYPE_Interbank credit_mean,bureau_CREDIT_TYPE_Interbank credit_sum,bureau_CREDIT_TYPE_Real estate loan_mean,bureau_CREDIT_TYPE_Real estate loan_sum,...,bureau_CREDIT_TYPE_Credit card_sum,bureau_CREDIT_ACTIVE_Active_mean,bureau_CREDIT_ACTIVE_Closed_mean,bureau_CREDIT_TYPE_Consumer credit_mean,bureau_CREDIT_CURRENCY_currency 1_mean,bureau_CREDIT_ACTIVE_Active_sum,bureau_CREDIT_ACTIVE_Closed_sum,bureau_CREDIT_TYPE_Consumer credit_sum,bureau_CREDIT_CURRENCY_currency 1_sum,bureau_CREDIT_ACTIVE_Active_count
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,0,0.428571,0.571429,1.000000,1.0,3,4,7,7,7
100002,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,4,0.250000,0.750000,0.500000,1.0,2,6,4,8,8
100003,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,2,0.250000,0.750000,0.500000,1.0,1,3,2,4,4
100004,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,0,0.000000,1.000000,1.000000,1.0,0,2,2,2,2
100005,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,1,0.666667,0.333333,0.666667,1.0,2,1,2,3,3


In [30]:
# Calculate value count statistics for each `SK_ID_CURR` 
df_bureau_balance_agg = kf.agg_numeric(df_bureau_balance, parent_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
df_bureau_balance_agg.head()

,bureau_balance_MONTHS_BALANCE_sum,bureau_balance_MONTHS_BALANCE_min,bureau_balance_MONTHS_BALANCE_mean,bureau_balance_MONTHS_BALANCE_max,bureau_balance_MONTHS_BALANCE_count
SK_ID_BUREAU,,,,,
5001709,-4656,-96,-48.0,0,97
5001710,-3403,-82,-41.0,0,83
5001711,-6,-3,-1.5,0,4
5001712,-171,-18,-9.0,0,19
5001713,-231,-21,-10.5,0,22


In [31]:
# Counts of each type of status for each previous loan
df_bureau_balance_counts = kf.agg_categorical(df_bureau_balance, parent_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
df_bureau_balance_counts.head()

,bureau_balance_STATUS_4_mean,bureau_balance_STATUS_3_mean,bureau_balance_STATUS_2_mean,bureau_balance_STATUS_5_mean,bureau_balance_STATUS_4_sum,bureau_balance_STATUS_3_sum,bureau_balance_STATUS_2_sum,bureau_balance_STATUS_5_sum,bureau_balance_STATUS_1_mean,bureau_balance_STATUS_1_sum,bureau_balance_STATUS_0_mean,bureau_balance_STATUS_0_sum,bureau_balance_STATUS_X_mean,bureau_balance_STATUS_C_mean,bureau_balance_STATUS_X_sum,bureau_balance_STATUS_C_sum,bureau_balance_STATUS_0_count
SK_ID_BUREAU,,,,,,,,,,,,,,,,,
5001709,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.000000,0,0.113402,0.886598,11,86,97
5001710,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.060241,5,0.361446,0.578313,30,48,83
5001711,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.750000,3,0.250000,0.000000,1,0,4
5001712,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.526316,10,0.000000,0.473684,0,9,19
5001713,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.000000,0,1.000000,0.000000,22,0,22


In [32]:
# Dataframe grouped by the loan
df_bureau_by_loan = df_bureau_balance_agg.merge(df_bureau_balance_counts, right_index = True, left_on = 'SK_ID_BUREAU', 
                                                how = 'outer')

# Merge to include the SK_ID_CURR
df_bureau_by_loan = df_bureau[['SK_ID_BUREAU', 'SK_ID_CURR']].merge(df_bureau_by_loan, on = 'SK_ID_BUREAU', how = 'left')

df_bureau_by_loan.head()

,SK_ID_BUREAU,SK_ID_CURR,bureau_balance_MONTHS_BALANCE_sum,bureau_balance_MONTHS_BALANCE_min,bureau_balance_MONTHS_BALANCE_mean,bureau_balance_MONTHS_BALANCE_max,bureau_balance_MONTHS_BALANCE_count,bureau_balance_STATUS_4_mean,bureau_balance_STATUS_3_mean,bureau_balance_STATUS_2_mean,...,bureau_balance_STATUS_5_sum,bureau_balance_STATUS_1_mean,bureau_balance_STATUS_1_sum,bureau_balance_STATUS_0_mean,bureau_balance_STATUS_0_sum,bureau_balance_STATUS_X_mean,bureau_balance_STATUS_C_mean,bureau_balance_STATUS_X_sum,bureau_balance_STATUS_C_sum,bureau_balance_STATUS_0_count
0,5714462,215354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5714463,215354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5714464,215354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5714465,215354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5714466,215354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# Aggregate the stats for each client
df_bureau_balance_by_client = kf.agg_numeric(df_bureau_by_loan.drop(columns = ['SK_ID_BUREAU']), parent_var = 'SK_ID_CURR', 
                                             df_name = 'client')
df_bureau_balance_by_client.head()

,client_bureau_balance_MONTHS_BALANCE_sum_sum,client_bureau_balance_MONTHS_BALANCE_sum_min,client_bureau_balance_MONTHS_BALANCE_sum_mean,client_bureau_balance_MONTHS_BALANCE_min_sum,client_bureau_balance_MONTHS_BALANCE_mean_sum,client_bureau_balance_MONTHS_BALANCE_min_min,client_bureau_balance_MONTHS_BALANCE_mean_min,client_bureau_balance_MONTHS_BALANCE_min_mean,client_bureau_balance_MONTHS_BALANCE_mean_mean,client_bureau_balance_MONTHS_BALANCE_sum_max,...,client_bureau_balance_STATUS_C_sum_mean,client_bureau_balance_MONTHS_BALANCE_count_mean,client_bureau_balance_STATUS_0_count_mean,client_bureau_balance_STATUS_X_sum_sum,client_bureau_balance_STATUS_0_sum_sum,client_bureau_balance_STATUS_C_sum_max,client_bureau_balance_MONTHS_BALANCE_count_max,client_bureau_balance_STATUS_0_count_max,client_bureau_balance_STATUS_C_sum_sum,client_bureau_balance_MONTHS_BALANCE_count_sum
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-2800.0,-1326.0,-400.000000,-165.0,-82.5,-51.0,-25.5,-23.571429,-11.785714,-1.0,...,15.714286,24.571429,24.571429,30.0,31.0,44.0,52.0,52.0,110.0,172.0
100002,-2701.0,-632.0,-337.625000,-226.0,-175.0,-47.0,-39.5,-28.250000,-21.875000,-6.0,...,2.875000,13.750000,13.750000,15.0,45.0,13.0,22.0,22.0,23.0,110.0
100003,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0
100004,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0
100005,-91.0,-78.0,-30.333333,-18.0,-9.0,-12.0,-6.0,-6.000000,-3.000000,-3.0,...,1.666667,7.000000,7.000000,2.0,14.0,5.0,13.0,13.0,5.0,21.0


In [34]:
original_features = list(df_application_train.columns)
print('Original Number of Features: ', len(original_features))

Original Number of Features:  122


In [35]:
# Merge with the value counts of bureau
df_application_train = df_application_train.merge(df_bureau_counts, on = 'SK_ID_CURR', how = 'left')

# Merge with the stats of bureau
df_application_train = df_application_train.merge(df_bureau_agg, on = 'SK_ID_CURR', how = 'left')

# Merge with the monthly information grouped by client
df_application_train = df_application_train.merge(df_bureau_balance_by_client, on = 'SK_ID_CURR', how = 'left')

In [36]:
new_features = list(df_application_train.columns)
print('Number of features using previous loans from other institutions data: ', len(new_features))

Number of features using previous loans from other institutions data:  313


## Autres tables

### Table previous_application

In [37]:
# Calculate aggregate statistics for each numeric column
df_previous_agg = kf.agg_numeric(df_prev_app, 'SK_ID_CURR', 'previous')
print('Previous aggregation shape: ', df_previous_agg.shape)
df_previous_agg.head()

Previous aggregation shape:  (338857, 80)


,previous_DAYS_DECISION_sum,previous_DAYS_DECISION_min,previous_DAYS_DECISION_mean,previous_DAYS_DECISION_max,previous_DAYS_FIRST_DUE_sum,previous_DAYS_FIRST_DUE_min,previous_DAYS_FIRST_DUE_mean,previous_DAYS_FIRST_DUE_max,previous_DAYS_LAST_DUE_sum,previous_DAYS_LAST_DUE_min,...,previous_DAYS_FIRST_DRAWING_min,previous_DAYS_FIRST_DRAWING_mean,previous_DAYS_FIRST_DRAWING_max,previous_DAYS_FIRST_DRAWING_sum,previous_RATE_INTEREST_PRIMARY_min,previous_RATE_INTEREST_PRIMARY_mean,previous_RATE_INTEREST_PRIMARY_max,previous_RATE_INTEREST_PRIVILEGED_min,previous_RATE_INTEREST_PRIVILEGED_mean,previous_RATE_INTEREST_PRIVILEGED_max
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-1740,-1740,-1740.0,-1740,-1709.0,-1709.0,-1709.000000,-1709.0,-1619.0,-1619.0,...,365243.0,365243.0,365243.0,365243.0,NaN,NaN,NaN,NaN,NaN,NaN
100002,-606,-606,-606.0,-606,-565.0,-565.0,-565.000000,-565.0,-25.0,-25.0,...,365243.0,365243.0,365243.0,365243.0,NaN,NaN,NaN,NaN,NaN,NaN
100003,-3915,-2341,-1305.0,-746,-3823.0,-2310.0,-1274.333374,-716.0,-3163.0,-1980.0,...,365243.0,365243.0,365243.0,1095729.0,NaN,NaN,NaN,NaN,NaN,NaN
100004,-815,-815,-815.0,-815,-784.0,-784.0,-784.000000,-784.0,-724.0,-724.0,...,365243.0,365243.0,365243.0,365243.0,NaN,NaN,NaN,NaN,NaN,NaN
100005,-1072,-757,-536.0,-315,-706.0,-706.0,-706.000000,-706.0,-466.0,-466.0,...,365243.0,365243.0,365243.0,365243.0,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
# Calculate value counts for each categorical column
df_previous_counts = kf.agg_categorical(df_prev_app, 'SK_ID_CURR', 'previous')
print('Previous counts shape: ', df_previous_counts.shape)
df_previous_counts.head()

Previous counts shape:  (338857, 285)


,previous_NAME_GOODS_CATEGORY_Animals_mean,previous_NAME_GOODS_CATEGORY_Animals_sum,previous_NAME_GOODS_CATEGORY_House Construction_mean,previous_NAME_GOODS_CATEGORY_House Construction_sum,previous_NAME_CASH_LOAN_PURPOSE_Refusal to name the goal_mean,previous_NAME_CASH_LOAN_PURPOSE_Refusal to name the goal_sum,previous_NAME_CASH_LOAN_PURPOSE_Money for a third person_mean,previous_NAME_CASH_LOAN_PURPOSE_Money for a third person_sum,previous_NAME_CASH_LOAN_PURPOSE_Hobby_mean,previous_NAME_CASH_LOAN_PURPOSE_Hobby_sum,...,previous_CODE_REJECT_REASON_XAP_mean,previous_FLAG_LAST_APPL_PER_CONTRACT_Y_mean,previous_NAME_PORTFOLIO_POS_sum,previous_NAME_CONTRACT_TYPE_Consumer loans_sum,previous_NAME_CASH_LOAN_PURPOSE_XAP_sum,previous_NAME_PRODUCT_TYPE_XNA_sum,previous_NAME_CONTRACT_STATUS_Approved_sum,previous_CODE_REJECT_REASON_XAP_sum,previous_FLAG_LAST_APPL_PER_CONTRACT_Y_sum,previous_NAME_CONTRACT_TYPE_Cash loans_count
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,1.0,1.0,1,1,1,1,1,1,1,1
100002,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,1.0,1.0,1,1,1,1,1,1,1,1
100003,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,1.0,1.0,2,2,2,2,3,3,3,3
100004,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,1.0,1.0,1,1,1,1,1,1,1,1
100005,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,...,1.0,1.0,1,1,1,2,1,2,2,2


In [39]:
# Merge in the previous information
df_application_train = df_application_train.merge(df_previous_counts, on ='SK_ID_CURR', how = 'left')
df_application_train = df_application_train.merge(df_previous_agg, on = 'SK_ID_CURR', how = 'left')

df_application_test = df_application_test.merge(df_previous_counts, on ='SK_ID_CURR', how = 'left')
df_application_test = df_application_test.merge(df_previous_agg, on = 'SK_ID_CURR', how = 'left')

# Remove variables to free memory
# gc.enable()
# del previous, previous_agg, previous_counts
# gc.collect()

In [40]:
df_application_train

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,previous_DAYS_FIRST_DRAWING_min,previous_DAYS_FIRST_DRAWING_mean,previous_DAYS_FIRST_DRAWING_max,previous_DAYS_FIRST_DRAWING_sum,previous_RATE_INTEREST_PRIMARY_min,previous_RATE_INTEREST_PRIMARY_mean,previous_RATE_INTEREST_PRIMARY_max,previous_RATE_INTEREST_PRIVILEGED_min,previous_RATE_INTEREST_PRIVILEGED_mean,previous_RATE_INTEREST_PRIVILEGED_max
0,100002,True,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,365243.0,365243.0,365243.0,365243.0,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,False,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,365243.0,365243.0,365243.0,1095729.0,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,False,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,365243.0,365243.0,365243.0,365243.0,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,False,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,365243.0,365243.0,365243.0,1460972.0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,False,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,365243.0,365243.0,365243.0,1826215.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,False,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,365243.0,365243.0,365243.0,365243.0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,False,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,365243.0,365243.0,365243.0,365243.0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,False,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,365243.0,365243.0,365243.0,730486.0,NaN,NaN,NaN,NaN,NaN,NaN
307509,456254,True,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,365243.0,365243.0,365243.0,730486.0,NaN,NaN,NaN,NaN,NaN,NaN


### Tables dépendantes de previous_application

In [41]:
df_cash_by_client = kf.aggregate_client(df_pos_cash_balance, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], df_names = ['cash', 'client'])
df_cash_by_client.head()

,client_cash_MONTHS_BALANCE_sum_sum,client_cash_MONTHS_BALANCE_min_sum,client_cash_MONTHS_BALANCE_mean_sum,client_cash_MONTHS_BALANCE_max_sum,client_cash_MONTHS_BALANCE_sum_min,client_cash_MONTHS_BALANCE_sum_mean,client_cash_MONTHS_BALANCE_sum_max,client_cash_MONTHS_BALANCE_min_min,client_cash_MONTHS_BALANCE_mean_min,client_cash_MONTHS_BALANCE_max_min,...,client_cash_CNT_INSTALMENT_FUTURE_max_sum,client_cash_NAME_CONTRACT_STATUS_Active_sum_sum,client_cash_CNT_INSTALMENT_min_sum,client_cash_CNT_INSTALMENT_mean_sum,client_cash_CNT_INSTALMENT_max_sum,client_cash_CNT_INSTALMENT_count_sum,client_cash_CNT_INSTALMENT_FUTURE_count_sum,client_cash_NAME_CONTRACT_STATUS_Active_count_sum,client_cash_CNT_INSTALMENT_FUTURE_sum_sum,client_cash_CNT_INSTALMENT_sum_sum
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-2887,-669,-653.0,-637,-378,-320.777778,-275,-96,-94.5,-93,...,28.0,32.0,36.0,36.000000,36.0,41,41,41,62.0,164.0
100002,-3610,-361,-190.0,-19,-190,-190.000000,-190,-19,-10.0,-1,...,456.0,361.0,456.0,456.000000,456.0,361,361,361,5415.0,8664.0
100003,-13240,-1348,-1226.0,-1104,-858,-472.857143,-172,-77,-71.5,-66,...,288.0,256.0,248.0,283.000000,288.0,272,272,272,1608.0,2840.0
100004,-408,-108,-102.0,-96,-102,-102.000000,-102,-27,-25.5,-24,...,16.0,12.0,12.0,15.000000,16.0,16,16,16,36.0,60.0
100005,-2420,-275,-220.0,-165,-220,-220.000000,-220,-25,-20.0,-15,...,132.0,99.0,99.0,128.699997,132.0,110,110,121,792.0,1287.0


In [42]:
df_application_train = df_application_train.merge(df_cash_by_client, on = 'SK_ID_CURR', how = 'left')

In [43]:
df_credit_by_client = kf.aggregate_client(df_credit_card_balance, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], 
                                       df_names = ['credit', 'client'])
df_credit_by_client.head()

,client_credit_MONTHS_BALANCE_sum_sum,client_credit_MONTHS_BALANCE_min_sum,client_credit_MONTHS_BALANCE_mean_sum,client_credit_MONTHS_BALANCE_sum_min,client_credit_MONTHS_BALANCE_sum_mean,client_credit_MONTHS_BALANCE_sum_max,client_credit_MONTHS_BALANCE_max_sum,client_credit_MONTHS_BALANCE_min_min,client_credit_MONTHS_BALANCE_min_mean,client_credit_MONTHS_BALANCE_min_max,...,client_credit_AMT_DRAWINGS_ATM_CURRENT_mean_max,client_credit_AMT_PAYMENT_CURRENT_mean_min,client_credit_AMT_PAYMENT_CURRENT_mean_mean,client_credit_AMT_PAYMENT_CURRENT_mean_max,client_credit_AMT_PAYMENT_CURRENT_max_min,client_credit_AMT_PAYMENT_CURRENT_max_mean,client_credit_AMT_PAYMENT_CURRENT_max_max,client_credit_AMT_DRAWINGS_ATM_CURRENT_max_min,client_credit_AMT_DRAWINGS_ATM_CURRENT_max_mean,client_credit_AMT_DRAWINGS_ATM_CURRENT_max_max
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100006,-126,-36,-21.0,-21,-21.0,-21,-6,-6,-6.0,-6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100011,-210826,-5550,-2849.0,-2849,-2849.0,-2849,-148,-75,-75.0,-75,...,2432.432373,4843.063965,4843.063965,4843.063965,55485.0,55485.0,55485.0,180000.0,180000.0,180000.0
100013,-446976,-9216,-4656.0,-4656,-4656.0,-4656,-96,-96,-96.0,-96,...,6350.000000,7168.346191,7168.346191,7168.346191,153675.0,153675.0,153675.0,157500.0,157500.0,157500.0
100021,-2890,-306,-170.0,-170,-170.0,-170,-34,-18,-18.0,-18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100023,-480,-88,-60.0,-60,-60.0,-60,-32,-11,-11.0,-11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
df_application_train = df_application_train.merge(df_credit_by_client, on = 'SK_ID_CURR', how = 'left')

In [45]:
df_installments_by_client = kf.aggregate_client(df_payments, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], 
                                                df_names = ['installments', 'client'])
df_installments_by_client.head()

,client_installments_DAYS_ENTRY_PAYMENT_sum_sum,client_installments_DAYS_INSTALMENT_sum_sum,client_installments_DAYS_ENTRY_PAYMENT_min_sum,client_installments_DAYS_INSTALMENT_min_sum,client_installments_DAYS_ENTRY_PAYMENT_mean_sum,client_installments_DAYS_INSTALMENT_mean_sum,client_installments_DAYS_ENTRY_PAYMENT_max_sum,client_installments_DAYS_INSTALMENT_max_sum,client_installments_DAYS_INSTALMENT_sum_min,client_installments_DAYS_ENTRY_PAYMENT_sum_min,...,client_installments_AMT_PAYMENT_min_sum,client_installments_AMT_INSTALMENT_min_sum,client_installments_AMT_PAYMENT_sum_max,client_installments_AMT_INSTALMENT_sum_max,client_installments_AMT_PAYMENT_mean_sum,client_installments_AMT_INSTALMENT_mean_sum,client_installments_AMT_INSTALMENT_max_sum,client_installments_AMT_PAYMENT_max_sum,client_installments_AMT_PAYMENT_sum_sum,client_installments_AMT_INSTALMENT_sum_sum
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-52813.0,-52598.0,-15608.0,-15584.0,-15365.0,-15314.0,-15080.0,-15044.0,-8658.0,-8647.0,...,2.774678e+04,2.774678e+04,2.925090e+04,2.925090e+04,4.119593e+04,4.119593e+04,8.153775e+04,8.153775e+04,1.528387e+05,1.528387e+05
100002,-113867.0,-106495.0,-11153.0,-10735.0,-5993.0,-5605.0,-931.0,-475.0,-5605.0,-5993.0,...,1.757837e+05,1.757837e+05,2.196257e+05,2.196257e+05,2.196257e+05,2.196257e+05,1.008781e+06,1.008781e+06,4.172888e+06,4.172888e+06
100003,-367137.0,-365546.0,-37757.0,-37514.0,-34633.0,-34454.0,-31594.0,-31394.0,-25740.0,-25821.0,...,1.154108e+06,1.154108e+06,1.150977e+06,1.150977e+06,1.618865e+06,1.618865e+06,4.394102e+06,4.394102e+06,1.134881e+07,1.134881e+07
100004,-6855.0,-6786.0,-2385.0,-2352.0,-2285.0,-2262.0,-2181.0,-2172.0,-2262.0,-2285.0,...,1.607175e+04,1.607175e+04,2.128846e+04,2.128846e+04,2.128846e+04,2.128846e+04,3.172189e+04,3.172189e+04,6.386539e+04,6.386539e+04
100005,-49374.0,-47466.0,-6624.0,-6354.0,-5486.0,-5274.0,-4230.0,-4194.0,-5274.0,-5486.0,...,4.331880e+04,4.331880e+04,5.616184e+04,5.616184e+04,5.616184e+04,5.616184e+04,1.589062e+05,1.589062e+05,5.054566e+05,5.054566e+05


In [46]:
df_application_train = df_application_train.merge(df_installments_by_client, on = 'SK_ID_CURR', how = 'left')

In [47]:
df_application_train = kf.remove_missing_columns(df_application_train)

There are 6 columns with greater than 90% missing values.


In [48]:
df_application_train.shape

(307511, 1324)